In [ ]:
import re
import sys
import numpy as np

import cadquery as cq
from cadquery.occ_impl.shapes import Wire

from OCP.TColgp import TColgp_Array1OfPnt
from OCP.gp import gp_Pnt
from OCP.Geom import Geom_BezierCurve
from OCP.BRepBuilderAPI import BRepBuilderAPI_MakeEdge, BRepBuilderAPI_MakeWire

from jupyter_cadquery import show, open_viewer
from jupyter_cadquery.viewer.client import _convert
from jupyter_cadquery import set_defaults
from jupyter_cadquery import Part, PartGroup


np.set_printoptions(threshold=sys.maxsize)

In [ ]:
svg1 = "M 18.6962 7.12238C 10.6836 7.12238 3.64131 4.24672 0 0C 1.41284 3.82041 3.96215 7.1163 7.30479 9.44404C 10.6474 11.7718 14.623 13.0196 18.6962 13.0196C 22.7695 13.0196 26.745 11.7718 30.0877 9.44404C 33.4303 7.1163 35.9796 3.82041 37.3925 4.0486e-13C 33.7601 4.24672 26.7445 7.12238 18.6962 7.12238Z"
svg2 = "M 18.6962 5.89725C 26.7089 5.89725 33.7512 8.77291 37.3925 13.0196C 35.9796 9.19922 33.4303 5.90333 30.0877 3.57559C 26.745 1.24785 22.7695 4.0486e-13 18.6962 0C 14.623 4.0486e-13 10.6474 1.24785 7.30479 3.57559C 3.96215 5.90333 1.41284 9.19922 0 13.0196C 3.64131 8.76401 10.648 5.89725 18.6962 5.89725Z"
dist = 2457.88 - 2484.27
color = "#F37726"

In [ ]:
class Move:
    def __init__(self, coords):
        self.x, self.y = [float(x) for x in coords.strip().split(" ")]

    def __repr__(self):
        return f"Move p=({self.x}, {self.y})"

    def convert(self):
        return None, gp_Pnt(self.x, self.y, 0.0)
    
class Curve:
    def __init__(self, coords):
        self.x1, self.y1, self.x2, self.y2, self.x, self.y = [float(x) for x in coords.strip().split(" ")]
        
    def convert(self, start):
        pole1 = gp_Pnt(self.x1, self.y1, 0.0)
        pole2 = gp_Pnt(self.x2, self.y2, 0.0)
        end = gp_Pnt(self.x, self.y, 0.0)
        array = TColgp_Array1OfPnt(1, 4)
        array.SetValue(1, start)
        array.SetValue(2, pole1)
        array.SetValue(3, pole2)
        array.SetValue(4, end)
        curve = Geom_BezierCurve(array)
        curve = BRepBuilderAPI_MakeEdge(curve).Edge()
        return curve, end
        
    def __repr__(self):
        return f"Curve p1=({self.x1}, {self.y1}), p2=({self.x2}, {self.y2}), p=({self.x}, {self.y})"

class Close:
    def __init__(self, coords):
        ...
        
    def convert(self, start):
        return start
    
    def __repr__(self):
        return "Close"
    
command_map = {
    "M": Move, "C": Curve, "Z": Close
}

def parse(svg):
    commands = re.split(r"([MCZ])", svg)
    if commands[-2] == "Z":
        commands = commands[1:]
    else:
        commands = commands[1:-1]
    commands = [command_map[c[0]](c[1]) for c in np.asarray(commands).reshape(-1,2)]
    return commands

def convert(curve):
    edges = []
    close = False
    for c in curve:
        if isinstance(c, Move):
            e, p = c.convert()
            start = e
        elif isinstance(c, Curve):
            e, p = c.convert(p)
            edges.append(e)
        elif isinstance(c, Close):
            close = True

    wire_builder = BRepBuilderAPI_MakeWire()

    for e in edges:
        wire_builder.Add(e)

    wire_builder.Build()

    if not wire_builder.IsDone():
        print("error")            
    else:
        wire = Wire(wire_builder.Wire())
        if close:
            return wire
        else:
            return wire

curve1 = parse(svg1)
edges1 = convert(curve1)
curve2 = parse(svg2)
edges2 = convert(curve2)
edges2 = edges2.translate((0, dist, 0))

top = cq.Workplane(edges1)
top.ctx.pendingWires.append(edges1)
top = top.extrude(2)

bottom = cq.Workplane(edges2)
bottom.ctx.pendingWires.append(edges2)
bottom = bottom.extrude(2)

jupyter = top.union(bottom).rotate((0,0,0), (1,0,0), 90)
c = jupyter.val().Center().toTuple()
jupyter = jupyter.translate((-c[0], -c[1], -c[2]))

cadquery = (
    cq.Workplane("XZ")
    .text("cq", 24, 2, combine='cut', halign="center", valign="top", font="roboto")
    .translate((-c[0]+17, -c[1], 5.2-c[2]))

)

logo = PartGroup([
    Part(jupyter, name="jupyter", color="#2980b9"),
    Part(cadquery, name="cadquery", color="#ffffff"),
], name="jupyter cadquery")


In [ ]:
params = dict(
#    position=[80.7, -144.2, 39.4], 
#    quaternion=[0.5993, 0.1576, 0.1964, 0.7599],
    position=[88.3, -118.6, 28.7], 
    quaternion=[0.6019, 0.2058, 0.2358, 0.7347],
    grid=(True, False, False), 
    ticks=3, 
    ortho=False,
    default_edge_color="#bbb",
)

In [ ]:
d = show(
    logo, 
    theme="light",
    **params
)

In [ ]:
import pickle
import json
import base64
from jupyter_cadquery.utils import Color

data = _convert(
    logo,
    **params
)

del data["config"]["default_color"]

msg = pickle.dumps(data, 4)
# msg = json.dumps(data, default=default)

with open("../jupyter_cadquery/logo.py", "w") as fd:
    fd.write("LOGO_DATA = '%s'" % base64.b64encode(msg).decode())
    # fd.write("LOGO_DATA = '%s'" % msg)

In [ ]:
# import numpy as np
# 
# def default(obj):
#     if type(obj).__module__ == np.__name__:
#         if isinstance(obj, np.ndarray):
#             return obj.tolist()
#         else:
#             return obj.item()
#     raise TypeError('Unknown type:', type(obj))
# 

In [ ]:
# def round6(obj):
#     if isinstance(obj, (np.ndarray, list, tuple)):
#         return [round6(tup) for tup in obj]
#     elif isinstance(obj, np.int32):
#         return obj.item()
#     else:
#         if isinstance(obj, (np.float32, np.float64)):
#             return round(obj.item(),6)
#         else:
#             return round(obj, 6)
#     
# for part in data["data"]["shapes"]["parts"]:
#     for array in ["vertices", "normals", "triangles", "edges"]:
#         part["shape"][array] = [round6(x) for x in part["shape"][array]]
#         
# #json.dumps(data, default=default)
# with open("../jupyter_cadquery/logo2.py", "w") as fd:
#     fd.write("LOGO_DATA = '%s'" % json.dumps(data, default=default))